## Análisis de factores
    - Es una tecnica de reducción de dimensionalidad utilizada, principalmente, para detectar la presencia de variables latentes.
    
    - El racional general detrás del método es que las variables observadas están correlacionadas entre sí, porque existe una variable latente a la que están correlacionadas. Es decir, la correlación entre las variables OBS_1 y OBS_2 no se debe a que éstas estén correlacionadas, sino que OBS_1 está correlacionada con LAT, lo mismo que para OBS_2 y LAT, en consecuencia, existe una correlación residual entre OBS_1 y OBS_2
    
    - La estimación de los scores y loadings es un proceso que involucra matrices, regresiones lineales e iteraciones entre ellos, por lo que no es ejecutado manualmente aquí, sino que se utiliza una librería
    
    - Los scores, que sí pueden ser calculados, no son usualmente el objetivo principal de este análisis, pues la solución de los sistemas de ecuaciones no es única, además de que pueden los factores pueden ser rotados para proyectar los datos originales de mejor manera.
    
    - Para mejorar la interpretabilidad de los pesos obtenidos se realiza una rotación de los Factores. En esta rotación se maximiza la varianza explicada por cada uno, con lo que se logra distinguir cuáles variables son mejor explicadas en cada factor.
    
    - Debido a que los resultados de este análisis pueden ser rotados, éstos ganan especial utilidad al interpretar cuáles son las variables de mayor peso en cada uno de los factores. Se tiene que ser cuidadoso al seleccionar el número de factores (debe ser menor al número de variables), debido a que éstos deberían tener un fundamento lógico de ser variables latentes detrás de las variables observadas.
    
    - La principal diferencia entre PCA y FA es que PCA rota los PC para capturar la mayor varianza posible en cada una, siendo así cómo se generan. Por su lado FA, es un análisis de correlaciones entre las variables observadas y latentes, cuyos factores pueden ser rotados para aislar mejor la aportación de cada variable.
    
    - Se sigue la siguiente notaciñon:
        X_j: Variable observada j[1,...,p]
        f_k: Factor común j[1,...,q]
        e_j: Factor específico k[1,...,p]
        lambda_jk: Coeficiente del factor k para la variable j

    - El FA intenta explicar las variables en términos de otras latentes, por lo que cada una de las variables observadas puede ser descrita como la combinación lineal de los factores comunes y específicos:
    
        X_j = sum[k in q](lambda_jk*f_k) + e_j
        
    - Debido a que f_k y e_j no tienen un valor único, estimar los scores de cada observación no siempre es realizado, por lo que el FA es útil para hacer un análisis de las varianzas explicadas por cada factor e identificar cuáles comparten un factor común:

        Var(X_j) = sum[k in q](lamda_jk^2) + Var(e_j)

            Nota: Es deseable utilizar variables estandarizadas, por lo que Var(X_j) = 1. Por lo que la suma de
            sum[k in q](lamda_jk^2) + Var(e_j) para todas las variables es igual a p.
    
    - Dentro de las condiciones de cómputo, se impone que la covarianza entre cualquier combinación de e_j y f_k sea cero, o sea, que se espera que sean independientes entre ellas.

    - Uno de los resultados principales que se obtienen son lambda_jk y var(e_j). Para el caso de lambda_jk, pueden analizarse en conjunto para una k en específico (un factor en específico) para determinar cuáles variables están mejor explicadas por éste. En el caso de Var(e_j), cada uno nos muestra la varianza que no pudo ser explicada por los q factores, por lo que un valor grande de este resultado implica que no hay una variable latente adecuada, por lo que se puede proceder a añadir un factor al análisis o eliminar la variable.
    
    - Para determinar que porcentaje de la varianza total es explicada por el factor k, se tiene que:
        
        porc_var_exp_f_k = sum[for j in p](lambda_jk)/p

            Nota: Recuerde que p = sum[for j in p][for k in q](lambda_jk) + sum[for j in p](Var(e_j))

### 1] Definir datos para el análisis

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets

# Numeric variables to run PCA
features = ["sep_len", "sep_wid", "pet_len", "pet_wid"]

# Load iris dataset
iris = datasets.load_iris()
df_data = pd.DataFrame(data=np.c_[iris["data"], iris["target"]], columns=features + ["target"])
df_aux_target = pd.DataFrame.from_dict({"target": [i for i in range(3)], "class": iris["target_names"]})
df_data = pd.merge(df_data, df_aux_target, on="target", how="left")
df_data.index.name = "index"

# Output
print("="*100)
print(df_data.head(5))
print("="*100)

       sep_len  sep_wid  pet_len  pet_wid  target   class
index                                                    
0          5.1      3.5      1.4      0.2     0.0  setosa
1          4.9      3.0      1.4      0.2     0.0  setosa
2          4.7      3.2      1.3      0.2     0.0  setosa
3          4.6      3.1      1.5      0.2     0.0  setosa
4          5.0      3.6      1.4      0.2     0.0  setosa


### 2] Correr FA (Factor Analysis)
    - Se consideran únicamente 2 factores, con lo que se logra explicar el 84.6% de la varianza total
    
    - Los valores lambda representan la carga de cada variable en el factor principal, se observa que en para el primer factor, existen valores altos con el mismo signo para sep_len, pet_len y pet_wid, por lo que estas variables se agrupan para definir una variable latente. Para el segundo factor, sep_wid es la que tiene un peso mayor, por lo que ésta variable es la que define mayormente la segunda variable latente.
    
    - Las unicidades reflejan que tan bien están explicadas las variables por los factores escogidos, se oberva que sep_wid tiene una unicidad de 0.519, por lo que el (1-.519)% de la varianza en esta variable no ha sido posible ser explicada por el modelo de dos factores. El resto de variables tiene valores adecuados.
    
    - El porcentaje de varianza explicada por el modelo de dos factores es de (0.64+.20)% = 84% de la varianza total, por lo que explicar el data set original a través de dos variables latentes es bastante adecuado.

In [48]:
seofrom sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler

# Transform data
scaler = StandardScaler()
scaler.fit(df_data[features])
trans_data = scaler.transform(df_data[features])

# Run PCA model and get PC projections
fa_model = FactorAnalysis(n_components=2, rotation="varimax")
fa_model.fit(trans_data)

#fa_model.noise_variance_
#(fa_model.components_**2).sum() + fa_model.noise_variance_.sum()

# Lambda values
lambdas = fa_model.components_

# Uniqueness - Var(e_j) - Uniqueness in each variable
var_e = fa_model.noise_variance_

# Variance explaines by each factor
var_f = (lambdas**2).sum(1)/( (lambdas**2).sum() + var_e.sum(0) )
var_f = np.append(var_f,[1-var_f.sum()])



print("="*100)
print("Lambda values:\n",np.round(lambdas,3))
print("\nUniqueness:\n", np.round(var_e,3))
print("\nPercentage of variance [f_k, e]: \n",np.round(var_f,3))
print("="*100)

Lambda values:
 [[ 0.986 -0.161  0.908  0.859]
 [-0.058 -0.674  0.417  0.438]]

Uniqueness:
 [0.024 0.519 0.001 0.071]

Percentage of variance [f_k, e]: 
 [0.64  0.206 0.154]
